## Working with QTM in Vgrid DGGS

[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.gishub.vn/lab/index.html?path=notebooks/vgrid/07_qtm.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeoshub/vgrid/blob/master)
[![image](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/opengeoshub/vgrid/HEAD)

Full Vgrid DGGS documentation is available at [vgrid document](https://vgrid.gishub.vn).

To work with Vgrid DGGS directly in GeoPandas and Pandas, please use [vgridpandas](https://pypi.org/project/vgridpandas/). Full Vgridpandas DGGS documentation is available at [vgridpandas document](https://vgridpandas.gishub.vn).

To work with Vgrid DGGS in QGIS, please use [Vgrid Plugin](https://plugins.qgis.org/plugins/vgridtools/).

### Install vgrid
Uncomment the following line to install [vgrid](https://pypi.org/project/vgrid/) if needed.

In [ ]:
# %pip install vgrid --upgrade

### latlon2qtm

In [ ]:
from vgrid.conversion.latlon2dggs import latlon2qtm

lat = 10.775276
lon = 106.706797
res = 14
qtm_id = latlon2qtm(lat, lon, res)
qtm_id

### QTM to Shapely Polygon

In [ ]:
from vgrid.conversion.dggs2geo.qtm2geo  import qtm2geo

qtm_geo = qtm2geo(qtm_id)
qtm_geo

### QTM to GeoJSON

In [ ]:
from vgrid.conversion.dggs2geo.qtm2geo import qtm2geojson

qtm_geojson = qtm2geojson(qtm_id)
qtm_geojson

### Vector to QTM

In [ ]:
from vgrid.conversion.vector2dggs.vector2qtm import vector2qtm

file_path = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/shape/polygon.geojson"
)
vector_to_qtm = vector2qtm(file_path, resolution=17, compact=False, 
                            predicate = "intersects", output_format="gpd")
# Visualize the output
vector_to_qtm.explore(tiles="CartoDB Positron")

### QTM Compact

In [ ]:
from vgrid.conversion.dggscompact import qtmcompact

qtm_compacted = qtmcompact(vector_to_qtm, qtm_id="qtm", output_format="gpd")
qtm_compacted.plot(edgecolor="white")

### QTM Expand

In [ ]:
from vgrid.conversion.dggscompact import qtmexpand

qtm_expanded = qtmexpand(vector_to_qtm, resolution=18, qtm_id="qtm", output_format="gpd")
qtm_expanded.plot(edgecolor="white")

### QTM Binning

In [ ]:
from vgrid.binning.qtmbin import qtmbin
import geopandas as gpd
file_path = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/csv/dist1_pois.csv"
)
stats = "count"
qtm_bin = qtmbin(file_path, resolution=16, stats=stats, 
                # numeric_field="confidence",
                # category="category",
                output_format="gpd")
qtm_bin.head()

### Raster to QTM

In [ ]:
# %pip install folium

In [ ]:
from vgrid.conversion.raster2dggs.raster2qtm import raster2qtm
from vgrid.utils.io import download_file          

raster_url = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/raster/rgb.tif"
)
raster_file = download_file(raster_url)

raster_to_qtm =  raster2qtm(raster_file,output_format="gpd")

# Visualize the output
import folium

m = folium.Map(tiles="CartoDB positron", max_zoom=28)

qtm_layer = folium.GeoJson(
    raster_to_qtm,
    style_function=lambda x: {
        "fillColor": f"rgb({x['properties']['band_1']}, {x['properties']['band_2']}, {x['properties']['band_3']})",
        "fillOpacity": 1,
        "color": "black",
        "weight": 1,
    },
    popup=folium.GeoJsonPopup(
        fields=["qtm", "band_1", "band_2", "band_3"],
        aliases=["QTM ID", "Band 1", "Band 2", "Band 3"],
        style="""
            background-color: white;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
    ),
).add_to(m)

m.fit_bounds(qtm_layer.get_bounds())

# Display the map
m

### QTM Generator

In [ ]:
from vgrid.generator.qtmgrid import qtmgrid
qtm_grid = qtmgrid(resolution=3,output_format="gpd")
# qtm_grid = qtmgrid(resolution=18,bbox=[106.699007, 10.762811, 106.717674, 10.778649],output_format="gpd")
qtm_grid.plot(edgecolor='white')

### QTM Statistics

In [ ]:
from vgrid.stats import qtmstats
qtmstats()